In [4]:
import pandas as pd
from scipy.stats import kruskal      # Importación de la función necesaria para estadística inferencial

In [ ]:
# Llamada a los datasets originales
df_flights = pd.read_csv("../data/Customer Flight Activity.csv")
df_loyalty = pd.read_csv("../data/Customer Loyalty History.csv")

In [6]:
# Reproducción de la transformación clave (agregación mensual)
df_flights_monthly = (
    df_flights
    .groupby(["Loyalty Number", "Year", "Month"], as_index=False)
    .sum()
)

In [8]:
# Reconstrucción df_merged
df_merged = df_flights_monthly.merge(
    df_loyalty,
    on="Loyalty Number",
    how="left"
)

# BONUS - ESTADÍSTICA INFERENCIAL

## OBJETIVO

Validar mediante una prueba de estadística inferencial si existen diferencial significativad en el número de vuelos reservados por mes según el nivel educativo de los clientes.

Este bonus en ningún caso sustituye el análisis principal sino que lo completa.

## CONTEXTO Y ALCANCE DEL ANÁLISIS

VARIABLES UTILIZADAS:

1.- Flights Booked: variable numérica discreta, que representa un conteo del número de vuelos reservados por cliente y mes, tomando únicamente valores enteros. Este tipo de variable suele presentar distribuciones asimétricas y presencia de ceros, lo que justifica el uso de pruebas no paramétricas, como la que se utiliza en este bonus.

2.- Education: variable categórica (niveles eductivos).

NIVEL DE ANÁLISIS:

1.- Registros mensuales por cliente, como en todo el ejercicio realizado.

2.- No se analizan totales acumulados.

## JUSTIFICACIÓN METODOLÓGICA

¿Por qué no ANOVA?

Flights Booked presenta: distribución asimétrica, muchos ceros y presencia de valores atípicos (outliers), por lo que no se cumple el supuesto de normalidad, y ANOVA no es apropiado.

¿Por qué Kruskal–Wallis?

Es una prueba no paramétrica que compara más de dos grupos independientes y no requiere normalidad, por lo que resulta adecuada para variables discretas y sesgadas, como es el caso. Flights Booked presenta una distribución sesgada a la derecha, con una alta concentración de valores bajos y una cola de valores elevados correspondientes a clientes muy activos. Este comportamiento es típico de variables de conteo y justifica el uso de métodos no paramétricos. Es la alternativa correcta a ANOVA en este contexto.

La prueba de Kruskal–Wallis permite evaluar si las distribuciones de los distintos grupos educativos difieren de forma significativa, sin asumir normalidad en los datos.

## FORMULACIÓN DE HIPÓTESIS

Hipótesis nula (H₀):
No existen diferencias estadísticamente significativas en el número de vuelos reservados por mes entre los distintos niveles educativos de los clientes.

Hipótesis alternativa (H₁):
Existen diferencias estadísticamente significativas en el número de vuelos reservados por mes en al menos uno de los niveles educativos.

Nivel de significación:
Se fija un nivel de significación α = 0.05.

## APLICACIÓN DE LA PRUEBA KRUSKAL–WALLIS

Se agrupan los valores de Flights Booked según el nivel educativo y se aplica la prueba de Kruskal–Wallis para evaluar si existen diferencias estadísticamente significativas entre los grupos.

In [9]:
df_edu_flights = (
    df_merged[["Education", "Flights Booked"]]
    .copy()
)

In [10]:
# Preparación de los grupos (arrays) separados
groups = [
    df_edu_flights.loc[df_edu_flights["Education"] == level, "Flights Booked"]   # para cada nivel filtra los vuelos 
    for level in df_edu_flights["Education"].unique()                            # los guarda en una lista de arrays, uno por grupo (unique)
]                       

In [ ]:
# Aplicación de la prueba Kruskal-Wallis
statistic, p_value = kruskal(*groups)        #  statistic: estadístico H de Kruskal; p_value: probabilidad asociada

statistic, p_value

(np.float64(18.71507053972487), np.float64(0.0008939855290647738))

In [12]:
# Decisión estadística (conexión con las hipótesis)
alpha = 0.05

if p_value < alpha:
    decision = "Se rechaza la hipótesis nula"
else:
    decision = "No se rechaza la hipótesis nula"

decision

'Se rechaza la hipótesis nula'

### RESUMEN TÉCNICO (NIVEL ESTADÍSTICO)

1.- Estadístico H (Kruskal) es de aprox. 18.72

2.- El p-value es de aprox. 0.00089, lo que indica que existen diferencias estadísticamente detectables entre las distribuciones de al menos dos niveles educativos. No obstante, como se observa en el análisis descriptivo y visual, dichas diferencias son pequeñas y no resultan relevantes desde el punto de vista práctico.

3.- α = 0.05

4.- Se rechaza la hipótesis nula, lo que indica que existen diferencias estadísticamente detectables entre al menos dos niveles educativos.

### LECTURA ANALÍTICA

La prueba de Kruskal–Wallis arroja un p-value inferior al nivel de significación establecido (α = 0.05), lo que indica que existen diferencias estadísticamente significativas en la distribución del número de vuelos reservados por mes entre al menos dos niveles educativos.

No obstante, estas diferencias deben interpretarse con cautela. El análisis descriptivo y visual previo mostró medias, medianas y dispersiones muy similares entre los grupos educativos, así como patrones de comportamiento homogéneos en los boxplots.

Dado el gran tamaño muestral, es posible que diferencias de magnitud reducida resulten estadísticamente significativas sin implicar un impacto relevante desde el punto de vista del negocio.

### INTERPRETACIÓN

Aunque la prueba estadística detecta diferencias significativas desde un punto de vista estadístico, los resultados no contradicen las conclusiones del análisis exploratorio. El nivel educativo no parece ser un factor determinante en el comportamiento mensual de reserva de vuelos, ya que las diferencias observadas son pequeñas y no estructurales.

Este análisis inferencial se incluye como validación adicional, reforzando la interpretación basada en análisis descriptivos y visuales, y subrayando la importancia de complementar la significación estadística con el criterio analítico y el contexto del negocio.

Este análisis inferencial confirma que, aunque estadísticamente se detectan diferencias entre grupos, el tamaño del efecto es reducido y no contradice las conclusiones prácticas del análisis descriptivo.

No se calcula explícitamente un tamaño del efecto; sin embargo, la similitud observada en medianas y rangos intercuartílicos sugiere que las diferencias detectadas tienen un impacto práctico reducido.